# 키워드 기반 이미지 크롤링 실습 (최소 100픽셀 필터링)

## 목표
- 사용자가 입력한 키워드로 Google 이미지 검색에서 사진 데이터를 크롤링
- 최소 한 변이 100픽셀 이상인 이미지만 다운로드
- 저장 경로: `'./data/수집데이터/[검색어]'`
- 전이학습 데이터셋 준비에 활용 가능

## 준비물
- Python 3.x
- 필요 라이브러리: `selenium`, `requests`, `beautifulsoup4`, `webdriver-manager`, `Pillow`
- 설치 명령어: `pip install selenium requests beautifulsoup4 webdriver-manager Pillow`
- Chrome 브라우저 설치

## 주의사항
- Google의 빈번한 요청 차단 가능성 있음 (대기 시간 조정 필요)
- 이미지 저작권 확인 후 학습 목적으로만 사용

## 실행 흐름
1. 환경 설정
2. 다운로드 함수 정의 (크기 필터링 포함)
3. 크롤링 함수 정의
4. 실행 및 다운로드

## 1. 환경 설정

In [ ]:
# !pip install webdriver_manager selenium beautifulsoup4 pillow

In [1]:
import os
import time
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from PIL import Image
import io

# 필요 라이브러리 확인
print('환경 설정 완료!')

환경 설정 완료!


## 2. 다운로드 함수 정의

- `create_download_folder`: `'./data/수집데이터/[검색어]'` 폴더 생성
- `is_image_large_enough`: 이미지 크기 확인 (최소 100픽셀)
- `download_image`: 이미지 다운로드 (크기 필터링 적용)

In [2]:
# 다운로드 폴더 생성 함수
def create_download_folder(keyword):
    base_path = '../data/수집데이터'
    folder_name = os.path.join(base_path, keyword)
    if not os.path.exists(folder_name):
        os.makedirs(folder_name, exist_ok=True)  # 상위 폴더도 생성
    return folder_name

# 이미지 크기 확인 함수
def is_image_large_enough(img_data, min_size=100):
    try:
        img = Image.open(io.BytesIO(img_data))
        width, height = img.size
        return width >= min_size or height >= min_size
    except Exception as e:
        print(f"이미지 크기 확인 오류: {str(e)}")
        return False

# 이미지 다운로드 함수 (크기 필터링 포함)
def download_image(img_url, folder, img_name):
    try:
        headers = {"User-Agent": "Mozilla/5.0"}
        response = requests.get(img_url, headers=headers, stream=True, timeout=10)
        if response.status_code == 200:
            img_data = response.content
            if is_image_large_enough(img_data):
                file_path = os.path.join(folder, img_name)
                with open(file_path, 'wb') as f:
                    f.write(img_data)
                print(f"다운로드 완료: {file_path}")
                return True
            else:
                print(f"이미지 크기 부족 (100픽셀 미만): {img_url}")
                return False
        else:
            print(f"다운로드 실패: {img_url} (상태 코드: {response.status_code})")
            return False
    except Exception as e:
        print(f"오류 발생: {img_url} - {str(e)}")
        return False

print('다운로드 함수 정의 완료!')

다운로드 함수 정의 완료!


## 3. 크롤링 함수 정의

- `crawl_images`: Google 이미지 검색을 크롤링하고 이미지 다운로드
- 최소 100픽셀 이상인 이미지만 저장

In [3]:
# Google 이미지 검색 크롤링 함수
def crawl_images(keyword, max_images=10):
    # Chrome 드라이버 설정
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # 브라우저 창 없이 실행
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    # Google 이미지 검색 URL
    search_url = f"https://www.google.com/search?q={keyword}&tbm=isch"
    driver.get(search_url)
    time.sleep(2)  # 페이지 로드 대기

    # 스크롤하여 더 많은 이미지 로드
    for _ in range(5):  # 5번 스크롤 (더 많은 이미지 확보)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

    # 페이지 소스 가져오기
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    img_tags = soup.find_all('img')

    # 다운로드 폴더 생성
    download_folder = create_download_folder(keyword)
    downloaded_count = 0

    # 이미지 URL 추출 및 다운로드
    for idx, img in enumerate(img_tags):
        if downloaded_count >= max_images:
            break
        img_url = img.get('src') or img.get('data-src')
        if img_url and img_url.startswith('http'):
            img_name = f"{keyword}_{downloaded_count+1}.jpg"
            if download_image(img_url, download_folder, img_name):
                downloaded_count += 1

    driver.quit()
    print(f"총 {downloaded_count}개의 이미지를 다운로드했습니다.")

print('크롤링 함수 정의 완료!')

크롤링 함수 정의 완료!


## 4. 실행 및 다운로드

- 키워드와 다운로드할 이미지 수를 입력하세요.
- 예: 'puppy', 5 입력 시 `'./data/수집데이터/puppy'`에 최소 100픽셀 이상인 이미지 5장 다운로드

In [7]:
# 사용자 입력 및 크롤링 실행
keyword = input("검색할 키워드를 입력하세요: ")
max_images = int(input("다운로드할 최대 이미지 수를 입력하세요 (기본값 10): ") or 10)
crawl_images(keyword, max_images)

이미지 크기 부족 (100픽셀 미만): https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSdYGUYFKe2-MpcyzgNgFH7HRWHsh_LG4nuqBedMrEvuYpJ9xTJMIuKL5cF&s
이미지 크기 확인 오류: cannot identify image file <_io.BytesIO object at 0x00000189CA833600>
이미지 크기 부족 (100픽셀 미만): https://fonts.gstatic.com/s/i/productlogos/googleg/v6/24px.svg
다운로드 완료: ../data/수집데이터\토끼\토끼_1.jpg
다운로드 완료: ../data/수집데이터\토끼\토끼_2.jpg
다운로드 완료: ../data/수집데이터\토끼\토끼_3.jpg
다운로드 완료: ../data/수집데이터\토끼\토끼_4.jpg
다운로드 완료: ../data/수집데이터\토끼\토끼_5.jpg
다운로드 완료: ../data/수집데이터\토끼\토끼_6.jpg
다운로드 완료: ../data/수집데이터\토끼\토끼_7.jpg
다운로드 완료: ../data/수집데이터\토끼\토끼_8.jpg
다운로드 완료: ../data/수집데이터\토끼\토끼_9.jpg
다운로드 완료: ../data/수집데이터\토끼\토끼_10.jpg
이미지 크기 부족 (100픽셀 미만): https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR_IcEEGiL75dix6-SLVQmz4Jo7lgWdQ1qt5kn3oPz8KPJ5oWtyCBxI&s=0
이미지 크기 부족 (100픽셀 미만): https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRxo_g2g-80BK4r3f37kcgcABUgc_SH1WsdciKz6y8V0HH_8zGExu3h3g&s=0
이미지 크기 부족 (100픽셀 미만): https://encrypted-tbn0.gstatic.com/images?q=tbn:A